<a href="https://colab.research.google.com/github/argenis-gomez/Clasificador-de-Tweets/blob/master/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fase 1: Importar las dependencias


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q tensorflow_text_nightly
!pip install -q tf-nightly

     |████████████████████████████████| 4.3MB 5.7MB/s 
     |████████████████████████████████| 456.7MB 24kB/s 
     |████████████████████████████████| 471kB 58.3MB/s 
     |████████████████████████████████| 4.0MB 27.7MB/s 
     |████████████████████████████████| 4.2MB 50.8MB/s 
     |████████████████████████████████| 5.9MB 43.1MB/s 
     |████████████████████████████████| 1.3MB 42.5MB/s 
     |████████████████████████████████| 3.9MB 38.9MB/s 
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.37.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.


In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup

import tensorflow as tf
from tensorflow.keras import layers
import zipfile
import os

from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import tensorflow_text as text

# Fase 2: Pre Procesado de Datos

## Carga de Ficheros

In [ ]:
tf.keras.utils.get_file("trainingandtestdata.zip",
                        "http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip",
                        cache_dir='.',
                        cache_subdir='')

with zipfile.ZipFile("trainingandtestdata.zip", 'r') as file:
  file.extractall("./")

os.rename("testdata.manual.2009.06.14.csv", "test.csv")
os.rename("training.1600000.processed.noemoticon.csv", "train.csv")

os.remove("trainingandtestdata.zip")

81371136/81363704 [==============================] - 6s 0us/step


In [ ]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
train_data = pd.read_csv(
    "train.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)
test_data = pd.read_csv(
    "test.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

In [ ]:
train_data.head()

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
train_data.sentiment.unique()

array([0, 4])

In [ ]:
test_data.head()

,sentiment,id,date,query,user,text
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...


In [ ]:
test_data.sentiment.unique()

array([4, 0, 2])

El conjunto de datos de testing tiene 3 etiquetas diferentes (una negativa, una positiva y una neutra), mientras que el conjunto de datos de entrenamiento tiene solo dos, por lo que no usaremos el archivo de testing y dividiremos el archivo de entrenamiento más tarde nosotros mismos.

In [ ]:
data = train_data

In [ ]:
data = data.sample(frac=1)
data.head()

,sentiment,id,date,query,user,text
240291,0,1980843418,Sun May 31 07:48:53 PDT 2009,NO_QUERY,SaraahPcd,@pcdnicole your present 'cause your birthday i...
266249,0,1988956829,Sun May 31 23:53:23 PDT 2009,NO_QUERY,jblockbuster,bought milk and coffee. Couldn't make foam wit...
331954,0,2012891451,Tue Jun 02 21:29:31 PDT 2009,NO_QUERY,tasherajean,@retrohandmade I finally found you guys. I ke...
278294,0,1991495677,Mon Jun 01 07:08:34 PDT 2009,NO_QUERY,bibibubut,"@DeniceSy :O DON'T TRUST ME ANYMORE, DEDE? I ..."
217802,0,1976014259,Sat May 30 16:24:05 PDT 2009,NO_QUERY,lmarconi,Boston's Hot Dog Safari at Suffolk Downs is to...


## Pre Procesado

### Limpieza

In [ ]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [ ]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    tweet = re.sub(r"www.[A-Za-z0-9./~]+", ' ', tweet)
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [ ]:
%%time

clean_data_path = '/content/drive/MyDrive/Clasificador Convolucional/clean_data.csv'

if not os.path.isfile(clean_data_path):
  
  data_clean = [clean_tweet(tweet) for tweet in data.text]

  data_labels = data.sentiment.values
  data_labels[data_labels == 4] = 1

  pd.DataFrame({'data': data_clean, 'labels': data_labels}).to_csv(clean_data_path, index=False)

else:
  print("Data limpia existente...")
  df = pd.read_csv(clean_data_path)
  data_clean = df.data.values
  data_labels = df.labels.values
  print("Restaurada", end='\n\n')

Data limpia existente...
Restaurada

CPU times: user 2.02 s, sys: 171 ms, total: 2.19 s
Wall time: 4.46 s


### Vocabulario

In [ ]:
def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)

In [ ]:
bert_tokenizer_params = dict()
reserved_tokens = ["[PAD]"]

bert_vocab_args = dict(
    vocab_size = 2**13,
    reserved_tokens=reserved_tokens,
    bert_tokenizer_params=bert_tokenizer_params,
    learn_params={},
)

In [ ]:
%%time

vocab_path = '/content/drive/MyDrive/Clasificador Convolucional/vocab.txt'

if not os.path.isfile(vocab_path):

  vocab = bert_vocab.bert_vocab_from_dataset(
      tf.data.Dataset.from_tensor_slices(data_clean).batch(1000).prefetch(tf.data.experimental.AUTOTUNE),
      **bert_vocab_args)

  write_vocab_file(vocab_path, vocab)

else:
  print('Vocabulario existente...', end='\n\n')

Vocabulario existente...

CPU times: user 1.2 ms, sys: 0 ns, total: 1.2 ms
Wall time: 1.52 ms


### Tokenización

In [ ]:
tokenizer = text.BertTokenizer(vocab_path)

In [ ]:
data_inputs = tokenizer.tokenize(data_clean)
data_inputs = data_inputs.merge_dims(-2,-1)

In [ ]:
data_inputs = data_inputs.to_list()

### Padding

In [ ]:
MAX_LEN = max([len(sentence) for sentence in data_inputs])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            padding="post",
                                                            maxlen=MAX_LEN)

### Dividimos en los conjuntos de entrenamiento y de testing


In [ ]:
split = int(len(data_inputs)*0.9)

train_inputs = data_inputs[:split]
train_labels = data_labels[:split]

test_inputs = data_inputs[split:]
test_labels = data_labels[split:]

In [ ]:
BATCH_SIZE = 1024

train_ds = tf.data.Dataset.from_tensor_slices((train_inputs, train_labels))
train_ds = train_ds.shuffle(len(train_inputs)).batch(BATCH_SIZE).cache().prefetch(tf.data.experimental.AUTOTUNE)

test_ds  = tf.data.Dataset.from_tensor_slices((test_inputs, test_labels))
test_ds  = test_ds.shuffle(len(test_inputs)).batch(BATCH_SIZE).cache().prefetch(tf.data.experimental.AUTOTUNE)

# Fase 3: Construción del modelo

In [ ]:
class DCNN(tf.keras.Model):
    
    def __init__(self, vocab_size, emb_dim=128, nb_filters=50, FFN_units=512, dropout_rate=0.1, name="dcnn"):
        super(DCNN, self).__init__(name=name)
        self.embedding = layers.Embedding(vocab_size, emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters, kernel_size=2, padding="valid", activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters, kernel_size=3, padding="valid", activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters, kernel_size=4, padding="valid", activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        self.dense = layers.Dense(units=FFN_units, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(2*dropout_rate))
        self.dropout = layers.Dropout(rate=dropout_rate)
        self.last_dense = layers.Dense(units=1, activation="sigmoid")
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1)
        merged = self.dense(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# Paso 4: Aplicación

## Configuración

In [ ]:
VOCAB_SIZE = 2**13

EMB_DIM = 64
NB_FILTERS = 100
FFN_UNITS = 256

DROPOUT_RATE = 0.4

NB_EPOCHS = 10

## Entrenamiento

In [ ]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            dropout_rate=DROPOUT_RATE)

Dcnn.compile(optimizer=tf.keras.optimizers.Adam(2e-6, 0.5),
             loss=tf.keras.losses.binary_crossentropy,
             metrics=["accuracy"])

In [ ]:
checkpoint_path = '/content/drive/MyDrive/Clasificador Convolucional/ckpt'

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=3)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

Último checkpoint restaurado!!


In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                              patience=3,
                                              restore_best_weights=True)

class CheckPoint(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
      ckpt_manager.save()
        
checkpoint = CheckPoint()

In [ ]:
Dcnn.fit(train_ds,
         validation_data=test_ds,
         epochs=NB_EPOCHS,
         callbacks=[early_stop, checkpoint])

Dcnn.save_weights('/content/drive/MyDrive/Clasificador Convolucional/ckpt/model_weights/model')

Epoch 1/10
1407/1407 [==============================] - 1802s 1s/step - loss: 0.3808 - accuracy: 0.8424 - val_loss: 0.4275 - val_accuracy: 0.8137
Epoch 2/10
1407/1407 [==============================] - 1839s 1s/step - loss: 0.3749 - accuracy: 0.8456 - val_loss: 0.4217 - val_accuracy: 0.8186
Epoch 3/10
1407/1407 [==============================] - 1839s 1s/step - loss: 0.3693 - accuracy: 0.8490 - val_loss: 0.4272 - val_accuracy: 0.8173
Epoch 4/10
1407/1407 [==============================] - 1832s 1s/step - loss: 0.3623 - accuracy: 0.8535 - val_loss: 0.4404 - val_accuracy: 0.8127
Epoch 5/10
1407/1407 [==============================] - 1801s 1s/step - loss: 0.3604 - accuracy: 0.8553 - val_loss: 0.4731 - val_accuracy: 0.8015


## Evaluación

In [ ]:
Dcnn.load_weights('/content/drive/MyDrive/Clasificador Convolucional/ckpt/model_weights/model')

In [ ]:
results = Dcnn.evaluate(test_inputs, test_labels, batch_size=BATCH_SIZE)
print(results)

157/157 [==============================] - 50s 321ms/step - loss: 0.4217 - accuracy: 0.8186
[0.4216708838939667, 0.8185937404632568]


In [ ]:
sentences = ["You are so funny",
             "i'm so happy",
             "hate this sommer",
             "This place is so boring",
             "You are so beautifull",
             "BA is the best place in world",
             "I like you best",
             "Espetial moment, bah! Was wonderfull, but i hate it."]

max_sen = max([len(sentence) for sentence in sentences]) + 1

sequence = tokenizer.tokenize(sentences)
sequence = sequence.merge_dims(-2,-1)
sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence.to_list(), padding="post", maxlen=MAX_LEN)

predictions = Dcnn(sequence, training=False).numpy()

for twit, prediction in zip(sentences, predictions):
  if prediction[0] > .5:
    sentiment = 'positive'
    value = prediction[0]*100
  else:
    sentiment = 'negative'
    value = (1 - prediction[0])*100

  print(f' - Tweet: {twit.ljust(max_sen, " ")} --> Sentiment: {value:2.2f}% {sentiment}', end='\n\n')
  

 - Tweet: You are so funny                                      --> Sentiment: 98.94% positive

 - Tweet: i'm so happy                                          --> Sentiment: 98.95% positive

 - Tweet: hate this sommer                                      --> Sentiment: 82.66% negative

 - Tweet: This place is so boring                               --> Sentiment: 95.11% negative

 - Tweet: You are so beautifull                                 --> Sentiment: 98.39% positive

 - Tweet: BA is the best place in world                         --> Sentiment: 96.68% positive

 - Tweet: I like you best                                       --> Sentiment: 98.14% positive

 - Tweet: Espetial moment, bah! Was wonderfull, but i hate it.  --> Sentiment: 57.69% negative

